In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# 데이터 압축풀기
!unzip /content/drive/MyDrive/lopen.zip
!mv open data

Archive:  /content/drive/MyDrive/lopen.zip
  inflating: open/sample_submission.csv  
  inflating: open/test.csv           
   creating: open/test/
  inflating: open/test/TEST_0000.mp4  
  inflating: open/test/TEST_0001.mp4  
  inflating: open/test/TEST_0002.mp4  
  inflating: open/test/TEST_0003.mp4  
  inflating: open/test/TEST_0004.mp4  
  inflating: open/test/TEST_0005.mp4  
  inflating: open/test/TEST_0006.mp4  
  inflating: open/test/TEST_0007.mp4  
  inflating: open/test/TEST_0008.mp4  
  inflating: open/test/TEST_0009.mp4  
  inflating: open/test/TEST_0010.mp4  
  inflating: open/test/TEST_0011.mp4  
  inflating: open/test/TEST_0012.mp4  
  inflating: open/test/TEST_0013.mp4  
  inflating: open/test/TEST_0014.mp4  
  inflating: open/test/TEST_0015.mp4  
  inflating: open/test/TEST_0016.mp4  
  inflating: open/test/TEST_0017.mp4  
  inflating: open/test/TEST_0018.mp4  
  inflating: open/test/TEST_0019.mp4  
  inflating: open/test/TEST_0020.mp4  
  inflating: open/test/TEST_0021.m

In [11]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore') 

In [12]:
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [14]:
# Hyperparameter Setting
CFG = {
    'VIDEO_LENGTH':50, # 10프레임 * 5초
    'IMG_SIZE':128,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':4,
    'SEED':41
}

In [15]:
# Fixed RandomSeed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [16]:
# 0 : Non-Crash 1 : Ego-Involve Yes 2 : Ego-Involve No
def crash_transform(label):
  if label == 0 :
    label = 0
  elif label > 7:
    label = 1
  else:
    label = 2
  return label

# 0 : Normal 1 : Snowy 2 : Rainy
def weather_transform(label):
  if label in [1,2,7,8]:
    label = 0
  elif label in [3,4,9,10]:
    label = 1
  elif label in [5,6,11,12]:
    label = 2
  return label

In [17]:
# Data Load
df = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [18]:
# train set의 video_path 수정
df['video_path'] = df['video_path'].apply(lambda x: os.path.join('./data', x[2:]))
df['video_path'].head()

0    ./data/train/TRAIN_0000.mp4
1    ./data/train/TRAIN_0001.mp4
2    ./data/train/TRAIN_0002.mp4
3    ./data/train/TRAIN_0003.mp4
4    ./data/train/TRAIN_0004.mp4
Name: video_path, dtype: object

In [19]:
# test set의 video_path 수정
test['video_path'] = test['video_path'].apply(lambda x: os.path.join('./data', x[2:]))
test['video_path'].head()

0    ./data/test/TEST_0000.mp4
1    ./data/test/TEST_0001.mp4
2    ./data/test/TEST_0002.mp4
3    ./data/test/TEST_0003.mp4
4    ./data/test/TEST_0004.mp4
Name: video_path, dtype: object

In [20]:
# 0 : Non-Crash 1 : Ego-Involve Yes 2 : Ego-Involve No
crash_ego_df = df.copy()
crash_ego_df['label'] = crash_ego_df['label'].apply(lambda label : crash_transform(label))

# Non-crash 데이터 제외
# 0 : day 1 : night
timing_df = df.loc[df['label']!=0]
timing_df['label'] = timing_df['label'].apply(lambda label : 0 if label in [1, 3, 5, 7, 9, 11] else 1)

# Non-crash 데이터 제외
# 0 : Normal 1 : Snowy 2 : Rainy
weather_df = df.loc[df['label']!=0]
weather_df['label'] = weather_df['label'].apply(lambda label : weather_transform(label))

In [21]:
# Train / Validation Split
crash_ego_train_set, crash_ego_val_set, _, _ = train_test_split(crash_ego_df,
                                                                crash_ego_df['label'],
                                                                test_size=0.1,
                                                                random_state=CFG['SEED'])

timing_train_set, timing_val_set, _, _ = train_test_split(timing_df,
                                                          timing_df['label'],
                                                          test_size=0.2,
                                                          random_state=CFG['SEED'])

weather_train_set, weather_val_set, _, _ = train_test_split(weather_df,
                                                            weather_df['label'],
                                                            test_size=0.2,
                                                            random_state=CFG['SEED'])

In [22]:
print('Crash+ego_involve\n', crash_ego_train_set['label'].value_counts(), '\n', crash_ego_val_set['label'].value_counts())
print('Train 데이터 비율:', crash_ego_train_set['label'].value_counts().sum()/(crash_ego_train_set['label'].value_counts().sum()+crash_ego_val_set['label'].value_counts().sum()))
print('Val 데이터 비율:', crash_ego_val_set['label'].value_counts().sum()/(crash_ego_train_set['label'].value_counts().sum()+crash_ego_val_set['label'].value_counts().sum()),'\n')

print('Weather\n', weather_train_set['label'].value_counts(), '\n', weather_val_set['label'].value_counts())
print('Train 데이터 비율:', weather_train_set['label'].value_counts().sum()/(weather_train_set['label'].value_counts().sum()+weather_val_set['label'].value_counts().sum()))
print('Val 데이터 비율:', weather_val_set['label'].value_counts().sum()/(weather_train_set['label'].value_counts().sum()+weather_val_set['label'].value_counts().sum()),'\n')

print('Timing\n', timing_train_set['label'].value_counts(), '\n', timing_val_set['label'].value_counts())
print('Train 데이터 비율:', timing_train_set['label'].value_counts().sum()/(timing_train_set['label'].value_counts().sum()+timing_val_set['label'].value_counts().sum()))
print('Val 데이터 비율:', timing_val_set['label'].value_counts().sum()/(timing_train_set['label'].value_counts().sum()+timing_val_set['label'].value_counts().sum()))

Crash+ego_involve
 0    1603
2     695
1     115
Name: label, dtype: int64 
 0    190
2     69
1     10
Name: label, dtype: int64
Train 데이터 비율: 0.8997017151379567
Val 데이터 비율: 0.10029828486204326 

Weather
 0    488
1    157
2     66
Name: label, dtype: int64 
 0    128
1     31
2     19
Name: label, dtype: int64
Train 데이터 비율: 0.7997750281214848
Val 데이터 비율: 0.20022497187851518 

Timing
 0    627
1     84
Name: label, dtype: int64 
 0    161
1     17
Name: label, dtype: int64
Train 데이터 비율: 0.7997750281214848
Val 데이터 비율: 0.20022497187851518


In [23]:
#CustomDataset

def cv2_to_pil(image):
    return Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

class CustomDataset1(Dataset):
    def __init__(self, video_path_list, label_list, tfms):
        self.video_path_list = video_path_list
        self.label_list = label_list
        self.tfms = tfms
        
    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])
        
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        
    def __len__(self):
        return len(self.video_path_list)
    
    def get_video(self, path):
        frames = []
        cap = cv2.VideoCapture(path)
        for _ in range(CFG['VIDEO_LENGTH']):
            _, img = cap.read()
            img = cv2.resize(img,(CFG["IMG_SIZE"], CFG["IMG_SIZE"]), interpolation = cv2.INTER_AREA)
            img = cv2_to_pil(img)
            img = (self.tfms(img))
            frames.append(img)
        frames = torch.from_numpy(np.stack(frames))
        return torch.FloatTensor(np.array(frames)).permute(1,0,2,3)

train_transform = Compose([
                 ToTensor(), 
                 Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989])])

In [24]:
class CustomDataset2(Dataset):
    def __init__(self, video_path_list, label_list, tfms):
        self.video_path_list = video_path_list
        self.label_list = label_list
        self.tfms = tfms
        
    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])
        
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        
    def __len__(self):
        return len(self.video_path_list)
    
    def get_video(self, path):
        frames = []
        cap = cv2.VideoCapture(path)
        for _ in range(CFG['VIDEO_LENGTH']):
            _, img = cap.read()
            img = cv2.resize(img,(200, 200), interpolation = cv2.INTER_AREA)
            img = cv2_to_pil(img)
            img = (self.tfms(img))
            frames.append(img)
        frames = torch.from_numpy(np.stack(frames))
        return torch.FloatTensor(np.array(frames)).permute(1,0,2,3)

train_transform = Compose([
                 ToTensor(), 
                 Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989])])

In [25]:
crash_ego_train_dataset = CustomDataset1(crash_ego_train_set['video_path'].values, crash_ego_train_set['label'].values,tfms = train_transform)
crash_ego_train_loader = DataLoader(crash_ego_train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
crash_ego_val_dataset = CustomDataset1(crash_ego_val_set['video_path'].values, crash_ego_val_set['label'].values,tfms = train_transform)
crash_ego_val_loader = DataLoader(crash_ego_val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

timing_train_dataset = CustomDataset2(timing_train_set['video_path'].values, timing_train_set['label'].values,tfms = train_transform)
timing_train_loader = DataLoader(timing_train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
timing_val_dataset = CustomDataset2(timing_val_set['video_path'].values, timing_val_set['label'].values,tfms = train_transform)
timing_val_loader = DataLoader(timing_val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

weather_train_dataset = CustomDataset2(weather_train_set['video_path'].values, weather_train_set['label'].values,tfms = train_transform)
weather_train_loader = DataLoader(weather_train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
weather_val_dataset = CustomDataset2(weather_val_set['video_path'].values, weather_val_set['label'].values,tfms = train_transform)
weather_val_loader = DataLoader(weather_val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [26]:
# test data (img size: 128)
test_dataset = CustomDataset1(test['video_path'].values, None, tfms = train_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=4)
# test data (img size: 200)
w_test_dataset = CustomDataset2(test['video_path'].values, None, tfms = train_transform)
w_test_loader = DataLoader(w_test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=4)

In [27]:
class BaseModel2(nn.Module): # crash+ego_involve
    def __init__(self, num_classes=3):
        super(BaseModel2, self).__init__()
        self.feature_extract = models.video.r3d_18(pretrained=True)
        self.f1 = nn.Linear(400, 256)
        self.f3 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.feature_extract(x)
        x = x.view(batch_size, -1)
        x = F.relu(self.f1(x))
        x = self.dropout(x)
        x = self.f3(x)
        return x

In [28]:
class BaseModel3(nn.Module): # timing
    def __init__(self, num_classes=2):
        super(BaseModel3, self).__init__()
        self.feature_extract = models.video.r3d_18(pretrained=True)
        self.f1 = nn.Linear(400, 256)
        self.f3 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.feature_extract(x)
        x = x.view(batch_size, -1)
        x = F.relu(self.f1(x))
        x = self.dropout(x)
        x = self.f3(x)
        return x

In [29]:
class BaseModel4(nn.Module): # weather
    def __init__(self, num_classes=3):
        super(BaseModel4, self).__init__()
        self.feature_extract = models.video.r3d_18(pretrained=True)
        self.f1 = nn.Linear(400, 256)
        self.f3 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.feature_extract(x)
        x = x.view(batch_size, -1)
        x = F.relu(self.f1(x))
        x = self.dropout(x)
        x = self.f3(x)
        return x

In [30]:
#train

def train(model, optimizer, train_loader, val_loader, scheduler, device, best_val_score): # epoch: 1
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    Iter = round(len(crash_ego_train_set)/4)
    best_val_score = best_val_score
    best_model = model
    
    for epoch in range(1, 2):
        model.train() 
        train_loss = []
        bat = 1
        for videos, labels in tqdm(iter(train_loader)): 
            
            videos = videos.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            output = model(videos)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()
            bat += 1
            train_loss.append(loss.item())
            print(f"batch {bat}/{Iter}", end='\r')
            
        _val_loss, _val_score, _f1_ = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : {_val_score} [{_f1_:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_f1_)

        if best_val_score < _f1_:
            best_val_score = _f1_
            best_model = model
    return best_model, best_val_score

In [31]:
def train2(model, optimizer, train_loader, val_loader, scheduler,  device, best_val_score): # epoch: 2
    criterion = nn.CrossEntropyLoss().to(device)
    Iter = round(len(crash_ego_train_set)/4)
    best_val_score = best_val_score
    best_model = model
    
    for epoch in range(1,3):
        model.train()
        train_loss = []
        bat = 1
        for videos, labels in tqdm(iter(train_loader)): 
            
            videos = videos.to(device)
            labels = labels.to(device) 
            optimizer.zero_grad()
            output = model(videos)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            bat += 1
            train_loss.append(loss.item())
            print(f"batch {bat}/{Iter}", end='\r')
            
        _val_loss, _val_score, _f1_ = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : {_val_score} [{_f1_:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_f1_)

        if best_val_score < _f1_:
            best_val_score = _f1_
            best_model = model
    return best_model, best_val_score

In [32]:
def train3(model, optimizer, train_loader, val_loader, scheduler, device, best_val_score): # epoch: 3
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    Iter = round(len(crash_ego_train_set)/4)
    best_val_score = best_val_score
    best_model = model
    
    for epoch in range(1, 4):
        model.train()
        train_loss = []
        bat = 1
        for videos, labels in tqdm(iter(train_loader)): 
            
            videos = videos.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            output = model(videos)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            bat += 1
            train_loss.append(loss.item())
            print(f"batch {bat}/{Iter}", end='\r')
    
        _val_loss, _val_score, _f1_= validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : {_val_score} [{_f1_:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_f1_)

        if best_val_score < _f1_:
            best_val_score = _f1_
            best_model = model
    return best_model, best_val_score

In [33]:
def train5(model, optimizer, train_loader, val_loader, scheduler, device, best_val_score): # epoch: 5
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    Iter = round(len(crash_ego_train_set)/4)
    best_val_score = best_val_score
    best_model = model
    
    for epoch in range(1, 6):
        model.train()
        train_loss = []
        bat = 1
        for videos, labels in tqdm(iter(train_loader)): 
            
            videos = videos.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            output = model(videos)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            bat += 1
            train_loss.append(loss.item())
            print(f"batch {bat}/{Iter}", end='\r')
    
        _val_loss, _val_score, _f1_= validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : {_val_score} [{_f1_:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_f1_)

        if best_val_score < _f1_:
            best_val_score = _f1_
            best_model = model
    return best_model, best_val_score

In [34]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    bat = 1
    Iter = round(len(crash_ego_val_set)/4)
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)

            logit = model(videos)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            _val_loss = np.mean(val_loss)
            bat += 1
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
            if bat%10 == 0:
                print(f"batch {bat}/{Iter} Val Loss: {_val_loss}", end='\r')
            
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average=None)
    _f1_ = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score, _f1_

In [35]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
        
            logit = model(videos)
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [36]:
ml = BaseModel4()
ml.eval()
optimizer = torch.optim.Adam(params = ml.parameters(), lr = 1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, 
                                                       patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)
infer_ml, best_val_score = train(ml, optimizer, weather_train_loader, weather_val_loader, scheduler, device, 0)

Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to /root/.cache/torch/hub/checkpoints/r3d_18-b3b3357e.pth
100%|██████████| 127M/127M [00:02<00:00, 48.9MB/s]


  0%|          | 0/178 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
optimizer = torch.optim.Adam(params = infer_ml.parameters(), lr = 5e-5, weight_decay=1e-5)
infer_ml, best_val_score = train(infer_ml, optimizer, weather_train_loader, weather_val_loader, None, device, best_val_score)

In [ ]:
optimizer = torch.optim.Adam(params = infer_ml.parameters(), lr = 1e-5, weight_decay=1e-5)
infer_ml, best_val_score = train(infer_ml, optimizer, weather_train_loader, weather_val_loader, None, device, best_val_score)

In [ ]:
optimizer = torch.optim.Adam(params = infer_ml.parameters(), lr = 1e-6, weight_decay=1e-5)
infer_ml, best_val_score = train(infer_ml, optimizer, weather_train_loader, weather_val_loader, None, device, best_val_score)

In [ ]:
weather_preds = inference(infer_ml, test_loader, device)

In [ ]:
weather_result = pd.read_csv('./data/sample_submission.csv')
weather_result['label'] = weather_preds
weather_result.to_csv('./drive/MyDrive/result/weather1.csv')